In [1]:
import dspy
from dspy.evaluate import Evaluate
from dspy.datasets.hotpotqa import HotPotQA
from dspy.teleprompt import BootstrapFewShotWithRandomSearch
from compiler.utils import load_api_key

load_api_key('/mnt/ssd4/lm_compiler/secrets.toml')

gpt4o_mini = dspy.OpenAI('gpt-4o-mini', max_tokens=1000)
colbert = dspy.ColBERTv2(url='http://192.168.1.18:8893/api/search')
dspy.configure(lm=gpt4o_mini, rm=colbert)

/mnt/ssd4/lm_compiler/my_env/lib/python3.12/site-packages/pydantic/_internal/_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)
/mnt/ssd4/lm_compiler/my_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = HotPotQA(train_seed=1, train_size=200, eval_seed=2023, dev_size=50, test_size=0)
trainset = [x.with_inputs('question') for x in dataset.train[0:150]]
valset = [x.with_inputs('question') for x in dataset.train[150:200]]
devset = [x.with_inputs('question') for x in dataset.dev]

# show an example datapoint; it's just a question-answer pair
trainset[0]

Example({'question': 'At My Window was released by which American singer-songwriter?', 'answer': 'John Townes Van Zandt'}) (input_keys={'question'})

In [3]:
from dsp.utils.utils import deduplicate

class BasicMH(dspy.Module):
    def __init__(self, passages_per_hop=3):
        super().__init__()

        self.retrieve = dspy.Retrieve(k=passages_per_hop)
        self.generate_query = [dspy.ChainOfThought("context, question -> search_query") for _ in range(2)]
        self.generate_answer = dspy.ChainOfThought("context, question -> answer")

    def forward(self, question):
        context = []

        for hop in range(2):
            search_query = self.generate_query[hop](context=context, question=question).search_query
            passages = self.retrieve(search_query).passages
            context = deduplicate(context + passages)

        return self.generate_answer(context=context, question=question).copy(context=context)

In [4]:
agent = BasicMH(passages_per_hop=2)

In [5]:
# Set up an evaluator on the first 300 examples of the devset.
config = dict(num_threads=8, display_progress=True, display_table=5)
evaluate = Evaluate(devset=devset, metric=dspy.evaluate.answer_exact_match, **config)

evaluate(agent)

 		You are using the client GPT3, which will be removed in DSPy 2.6.
 		Changing the client is straightforward and will let you use new features (Adapters) that improve the consistency of LM outputs, especially when using chat LMs. 

 		Learn more about the changes and how to migrate at
 		https://github.com/stanfordnlp/dspy/blob/main/examples/migration.ipynb
 		You are using the client GPT3, which will be removed in DSPy 2.6.
 		Changing the client is straightforward and will let you use new features (Adapters) that improve the consistency of LM outputs, especially when using chat LMs. 

 		Learn more about the changes and how to migrate at
 		https://github.com/stanfordnlp/dspy/blob/main/examples/migration.ipynb
 		You are using the client GPT3, which will be removed in DSPy 2.6.
 		Changing the client is straightforward and will let you use new features (Adapters) that improve the consistency of LM outputs, especially when using chat LMs. 

 		Learn more about the changes and how to

,question,example_answer,gold_titles,rationale,pred_answer,context,answer_exact_match
0,Are both Cangzhou and Qionghai in the Hebei province of China?,no,"{'Cangzhou', 'Qionghai'}","determine the locations of Cangzhou and Qionghai. Cangzhou is explicitly stated to be in eastern Hebei province, while Qionghai is identified as a county-level city...","No, Cangzhou is in Hebei province, while Qionghai is in Hainan province.","['Cangzhou | ""Cangzhou Cangzhou () is a prefecture-level city in eastern Hebei province, People\'s Republic of China. At the 2010 census, Cangzhou\'s built-up (""""or metro"""")...",
1,Who conducts the draft in which Marc-Andre Fleury was drafted to the Vegas Golden Knights for the 2017-18 season?,National Hockey League,"{'2017 NHL Expansion Draft', '2017–18 Pittsburgh Penguins season'}",identify the organization responsible for conducting the draft. The context mentions that the 2017 NHL Expansion Draft was conducted by the National Hockey League (NHL)...,The National Hockey League (NHL) conducts the draft.,"['""Marc-André Fleury"" | ""route to a Stanley Cup championship victory, defeating the Nashville Predators in six games. The win was the third Stanley Cup of...",
2,"The Wings entered a new era, following the retirement of which Canadian retired professional ice hockey player and current general manager of the Tampa Bay...",Steve Yzerman,"{'2006–07 Detroit Red Wings season', 'Steve Yzerman'}",identify the retired professional ice hockey player mentioned in the context. We know that the question refers to a significant event in the history of...,Steve Yzerman,"['""Julien BriseBois"" | ""Julien BriseBois Julien BriseBois (born January 24, 1977) is a Canadian ice hockey executive. He is the general manager for the Tampa...",✔️ [True]
3,What river is near the Crichton Collegiate Church?,the River Tyne,"{'Crichton Castle', 'Crichton Collegiate Church'}","determine the river near the Crichton Collegiate Church. We know that Crichton Collegiate Church is located in Midlothian, Scotland, and from the context provided, we...",River Tyne,"['""Crichton Collegiate Church"" | ""Crichton Collegiate Church Crichton Collegiate Church is situated about south west of the hamlet of Crichton in Midlothian, Scotland. Crichton itself...",✔️ [True]
4,In the 10th Century A.D. Ealhswith had a son called Æthelweard by which English king?,King Alfred the Great,"{'Æthelweard (son of Alfred)', 'Ealhswith'}",determine the answer. We know from the context that Æthelweard was the younger son of King Alfred the Great and Ealhswith. Since the question specifically...,King Alfred the Great,"['""Æthelweard (son of Alfred)"" | ""Æthelweard (son of Alfred) Æthelweard (d. 920 or 922) was the younger son of King Alfred the Great and Ealhswith....",✔️ [True]


32.0

In [6]:
import dspy.evaluate
from dspy.teleprompt import BootstrapFewShotWithRandomSearch, MIPROv2
prmopt_model = dspy.OpenAI(model='gpt-4o-mini', max_tokens=1000)
dspy.settings.configure(lm=prmopt_model)
bootstrap_optimizer = MIPROv2(
    prompt_model=prmopt_model,
    metric=dspy.evaluate.answer_exact_match,
    num_candidates=20,
    num_threads=8,
)
optimized_matplot = bootstrap_optimizer.compile(
    agent,
    trainset=trainset,
    max_bootstrapped_demos=2,
    max_labeled_demos=2,
    num_trials=25,
    valset=valset,
    requires_permission_to_run=False,
)

Beginning MIPROv2 optimization process...

==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
These will be used for as few-shot examples candidates for our program and for creating instructions.

Bootstrapping N=20 sets of demonstrations...
Bootstrapping set 1/20
Bootstrapping set 2/20
Bootstrapping set 3/20


  5%|▍         | 7/150 [00:42<14:28,  6.08s/it]


Bootstrapped 2 full traces after 8 examples in round 0.
Bootstrapping set 4/20


  3%|▎         | 4/150 [00:25<15:32,  6.39s/it]


Bootstrapped 2 full traces after 5 examples in round 0.
Bootstrapping set 5/20


...

Score: 44.0 on minibatch of size 25 with parameters ['Predictor 1: Instruction 7', 'Predictor 1: Few-Shot Set 12', 'Predictor 2: Instruction 18', 'Predictor 2: Few-Shot Set 19', 'Predictor 3: Instruction 2', 'Predictor 3: Few-Shot Set 16'].


== Minibatch Trial 25 / 25 ==


Average Metric: 10 / 25  (40.0): 100%|██████████| 25/25 [00:19<00:00,  1.31it/s]


Score: 40.0 on minibatch of size 25 with parameters ['Predictor 1: Instruction 11', 'Predictor 1: Few-Shot Set 18', 'Predictor 2: Instruction 1', 'Predictor 2: Few-Shot Set 18', 'Predictor 3: Instruction 14', 'Predictor 3: Few-Shot Set 11'].


===== Full Eval 3 =====
Doing full eval on next top averaging program (Avg Score: 56.0) so far from mini-batch trials...


Average Metric: 24 / 50  (48.0): 100%|██████████| 50/50 [00:23<00:00,  2.11it/s]  

Full eval score: 48.0
Best full eval score so far: 48.0




In [8]:
evaluate(optimized_matplot)

  0%|          | 0/50 [00:00<?, ?it/s]

Average Metric: 22 / 50  (44.0): 100%|██████████| 50/50 [00:40<00:00,  1.24it/s]


,question,example_answer,gold_titles,rationale,pred_answer,context,answer_exact_match
0,Are both Cangzhou and Qionghai in the Hebei province of China?,no,"{'Cangzhou', 'Qionghai'}","determine the answer. The context provides information about Cangzhou, stating that it is a prefecture-level city in eastern Hebei province, People's Republic of China. However,...",no,"['Cangzhou | ""Cangzhou Cangzhou () is a prefecture-level city in eastern Hebei province, People\'s Republic of China. At the 2010 census, Cangzhou\'s built-up (""""or metro"""")...",✔️ [True]
1,Who conducts the draft in which Marc-Andre Fleury was drafted to the Vegas Golden Knights for the 2017-18 season?,National Hockey League,"{'2017 NHL Expansion Draft', '2017–18 Pittsburgh Penguins season'}","determine who conducts the draft. The context mentions the ""2017 NHL Expansion Draft,"" which was conducted by the National Hockey League (NHL) to fill the...",National Hockey League (NHL),"['""Marc-André Fleury"" | ""route to a Stanley Cup championship victory, defeating the Nashville Predators in six games. The win was the third Stanley Cup of...",
2,"The Wings entered a new era, following the retirement of which Canadian retired professional ice hockey player and current general manager of the Tampa Bay...",Steve Yzerman,"{'2006–07 Detroit Red Wings season', 'Steve Yzerman'}","determine the answer. The context provides information about Steve Yzerman, who is a Canadian former professional ice hockey player and served as the captain of...",Steve Yzerman,"['""Steve Yzerman"" | ""Steve Yzerman Stephen Gregory Yzerman (; born May 9, 1965) is a Canadian former professional ice hockey player who spent his entire...",✔️ [True]
3,What river is near the Crichton Collegiate Church?,the River Tyne,"{'Crichton Castle', 'Crichton Collegiate Church'}","determine the answer. The context provided does not mention any specific river associated with Crichton Collegiate Church. It primarily focuses on the church's location, its...",The context does not provide information about a river near Crichton Collegiate Church.,"['""Crichton Collegiate Church"" | ""Crichton Collegiate Church Crichton Collegiate Church is situated about south west of the hamlet of Crichton in Midlothian, Scotland. Crichton itself...",
4,In the 10th Century A.D. Ealhswith had a son called Æthelweard by which English king?,King Alfred the Great,"{'Æthelweard (son of Alfred)', 'Ealhswith'}","determine the answer. The context provides information about Ealhswith, who was the wife of King Alfred the Great. It states that Ealhswith and Alfred had...",King Alfred the Great,"['Ealhswith | ""is commemorated in two early tenth century manuscripts as """"the true and dear lady of the English"""". Ealhswith had a brother called Æthelwulf,...",✔️ [True]


44.0

In [9]:
optimized_matplot.save('optimized_qa.dspy')

[('retrieve', <dspy.retrieve.retrieve.Retrieve object at 0x7f7cb84ce2d0>), ('generate_query[0]', Predict(StringSignature(context, question -> rationale, search_query
    instructions='Given the fields `context`, `question`, produce the fields `search_query`.'
    context = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Context:', 'desc': '${context}'})
    question = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Question:', 'desc': '${question}'})
    rationale = Field(annotation=str required=True json_schema_extra={'prefix': "Reasoning: Let's think step by step in order to", 'desc': '${produce the search_query}. We ...', '__dspy_field_type': 'output'})
    search_query = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'output', 'prefix': 'Search Query:', 'desc': '${search_query}'})
))), ('generate_query[1]', Predict(StringSignature(context, question -> rationale,

In [ ]:
optimized_matplot('Are both Cangzhou and Qionghai in the Hebei province of China?')